In [ ]:
#q1
def Q1(observations, states, start_p, trans_p, emit_p):
    num_observations = len(observations)
    viterbi_matrix = {state: [0] * num_observations for state in states}
    backpointer = {state: [0] * num_observations for state in states}
  #initialisaion step
    for state in states:
        viterbi_matrix[state][0] = start_p[state] * emit_p[state][observations[0]]
        backpointer[state][0] = -1
#recursion step
    for t in range(1, num_observations):
        for state in states:
            max_prob, prev_state_best = max((viterbi_matrix[prev_state][t - 1] * trans_p[prev_state][state], prev_state) for prev_state in states)
            viterbi_matrix[state][t] = max_prob * emit_p[state][observations[t]]
            backpointer[state][t] = prev_state_best

    best_path_prob = -1
    best_last_state = None
    for state in states:
        path_prob = viterbi_matrix[state][-1]
        if path_prob > best_path_prob:
            best_path_prob = path_prob
            best_last_state = state

    best_path = [best_last_state]
    for t in range(num_observations - 1, 0, -1):
        best_path.insert(0, backpointer[best_path[0]][t])

    return viterbi_matrix, best_path_prob, best_path

def run_Q1(test_case):
    return Q1(**test_case)

def display_results(output, states):
    try:
        print("\na) Probability of Hidden States for all three steps:")
        for state in states:
            print(f"{state}: {output[0][state]}")
    except Exception as e:
        print("Error in printing hidden state probabilities:", str(e))

    try:
        print("\nb) Highest probability at step 3 from where we start backtracking:")
        print(output[1])
    except Exception as e:
        print("Error in printing highest probability at step 3:", str(e))

    try:
        print("\nc) Most probable set of hidden states for the above-given input:")
        print(output[2])
    except Exception as e:
        print("Error in printing most probable set of hidden states:", str(e))

def menu():
    test_case = {
        'observations': ("normal", "cold", "dizzy"),
        'states': ("Healthy", "Fever"),
        'start_p': {"Healthy": 0.6, "Fever": 0.4},
        'trans_p': {
            "Healthy": {"Healthy": 0.7, "Fever": 0.3},
            "Fever": {"Healthy": 0.4, "Fever": 0.6},
        },
        'emit_p': {
            "Healthy": {"normal": 0.5, "cold": 0.4, "dizzy": 0.1},
            "Fever": {"normal": 0.1, "cold": 0.3, "dizzy": 0.6},
        }
    }

    while True:
        print("\nMenu:")
        print("1: Run Q1 Algorithm")
        print("2: Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            output = run_Q1(test_case)
            display_results(output, test_case['states'])
        elif choice == '2':
            print("Exiting program.")
            break
        else:
            print("Invalid choice! Please enter 1 or 2.")

if __name__ == "__main__":
    menu()



Menu:
1: Run Q1 Algorithm
2: Exit
Enter your choice: 1

a) Probability of Hidden States for all three steps:
Healthy: [0.3, 0.084, 0.00588]
Fever: [0.04000000000000001, 0.027, 0.01512]

b) Highest probability at step 3 from where we start backtracking:
0.01512

c) Most probable set of hidden states for the above-given input:
['Healthy', 'Healthy', 'Fever']

Menu:
1: Run Q1 Algorithm
2: Exit
Enter your choice: 2
Exiting program.


In [ ]:
#q2
def hmm_forward_probability(sequence, states, initial_probabilities, transition_matrix, emission_matrix):
    length = len(sequence)
    F = [[0.0] * len(states) for _ in range(length)]

    for j, state in enumerate(states):
        F[0][j] = initial_probabilities[state] * emission_matrix[state][sequence[0]]
   #iteration step
    for i in range(1, length):
        for j, state in enumerate(states):
            F[i][j] = sum(F[i - 1][k] * transition_matrix[prev_state][state] for k, prev_state in enumerate(states)) * emission_matrix[state][sequence[i]]
  # Sum the probabilities of ending in any state to get the total probability of the sequence
    final_probability = sum(F[length - 1])
    return final_probability


def Q2(test_case):
    sequence, states, initial_probabilities, transition_matrix, emission_matrix = test_case
    return hmm_forward_probability(sequence, states, initial_probabilities, transition_matrix, emission_matrix)


test_case = (
    "xzyyzzyzyy",
    ["A", "B"],
    {"A": 0.5, "B": 0.5},
    {"A": {"A": 0.303, "B": 0.697}, "B": {"A": 0.831, "B": 0.169}},
    {"A": {"x": 0.533, "y": 0.065, "z": 0.402}, "B": {"x": 0.342, "y": 0.334, "z": 0.324}}
)

output = Q2(test_case)
print("Probability Pr(x):", output)


Probability Pr(x): 1.1005510319694847e-06


In [1]:
#q3 final_draft
import numpy as np
import collections as cls
from math import log,inf
def prob_calculation(alignment, columns,transition_prob, emission_prob):
  for line in alignment:
    i=-1
    idx=0
    for j in range(len(line)):
      if columns[j]==True:
        i=i+1
        if line[j]!="-":
          y=3*i+2 # for deletion
        else:
          y=3*i+3 # for match/substitution
        transition_prob[idx,y]=transition_prob[idx,y]+1
        idx=y
      else:
        if line[j]!="-":
          y=3*i+4 # for insertion
          transition_prob[idx,y]=transition_prob[idx,y]+1
          idx=y
      if line[j]!="-":
        emission_prob[y,alphabet.index(line[j])]+=1
    transition_prob[idx,y]=transition_prob[idx,y]+1
  with np.errstate(divide="ignore", invalid="ignore"): # to avoid zero division error
    ep=[]
    for row in emission_prob:
      s=(row/sum(row))
      s[row==0.0]=0.0
      ep.append(s)
    emission_prob=np.array(ep)

    tp=[]
    for row in transition_prob:
      s=(row/sum(row))
      s[row==0.0]=0.0
      tp.append(s)
    transition_prob=np.array(tp)

  # Now maintaining the pseudocount
  n=(emission_prob.shape[0]-3)//3
  emission_prob[1,:]=emission_prob[1,:]+1
  for x in range(n):
    emission_prob[3*x+2,:]=emission_prob[3*x+2,:]+sigma #adding psedocount to every state so that none will be at probability zero
    emission_prob[3*x+4,:]=emission_prob[3*x+4,:]+sigma
    with np.errstate(divide="ignore", invalid="ignore"):
      ep=[]
      for row in emission_prob:
        s=(row/sum(row))
        s[row==0.0]=0.0
        ep.append(s)
      emission_prob=np.array(ep)

  with np.errstate(divide="ignore", invalid="ignore"):
    ep=[]
    for row in emission_prob:
      s=(row/sum(row))
      s[row==0.0]=0.0
      ep.append(s)
    emission_prob=np.array(ep)

  n=(transition_prob.shape[0]-3)//3


  transition_prob[0,1:4]+=sigma
  transition_prob[1,1:4]+=sigma


  for x in range(n):
    transition_prob[3*i+2:3*i+5,3*i+4:3*i+7]+= sigma # maintaining pseducount in transition probability as well
  with np.errstate(divide="ignore", invalid="ignore"): #to avoid zero division error
    tp=[]
    for row in transition_prob:
      s=(row/sum(row))
      s[row==0.0]=0.0
      tp.append(s)
    transition_prob=np.array(tp)

    return emission_prob, transition_prob

def backtracking(n,m,node,column):


  if node[0]=="E":
    return [(f"D{n}",m),(f"M{n}",m),(f"I{n}",m)]

  i=int(node[1:])

  if node=="I0" or node =="M1":
    if column==1:
      return [("S",0)]
    else:
      return [("I0",column-1)]


  if column==0:
    if i==1:
      return [("S",0)]
    else:
      return [(f"D{i-1}",0)]

  elif node[0]=="I":
    if column==1:
      return [(f"D{i}",0)]
    else:
      return [(f"D{i}",column-1),(f"M{i}",column-1),(f"I{i}",column-1)]

  elif node[0]=="M":
    if column==1:
      return [(f"D{i-1}",0)]
    else:
      return [(f"D{i-1}",column-1),(f"M{i-1}",column-1),(f"I{i-1}",column-1)]

  elif node[0]=="D":
    if i==1:
      return [(f"I0",column)]
    else:
      return [(f"D{i-1}",column),(f"M{i-1}",column),(f"I{i-1}",column)]





def q3(text,theta,sigma,alignment,alphabet):

  columns=[(sum(seq[x]=="-" for seq in alignment)/len(alignment))<theta for x in range(len(alignment[0]))] #finding the valid columns having values less than threshold
  valid_col=sum(columns) #total count of v
  total_states=2+3*(valid_col) # total number of valid states
  transition_prob=np.zeros((3+3*valid_col,3+3*valid_col),dtype=float) #matrix to store transition probability between the states
  emission_prob=np.zeros((3+3*valid_col,len(alphabet)),dtype=float) #matrix to store emission probability from every states

  # Calculating transition and emission probability
  emission_prob, transition_prob= prob_calculation(alignment, columns, transition_prob, emission_prob)

  n=(emission_prob.shape[0]-3)//3

  # now storing transition and emission probability in a dictionary with respect to all the states
  # using default dictionary from collections to avoid key errors
  tp=cls.defaultdict(float)
  labels=["S"]
  labels.append("I0")
  for i in range(1,n+1):
    labels.append(f"M{i}")
    labels.append(f"D{i}")
    labels.append(f"I{i}")
  labels.append("E")
  r=transition_prob.shape[0]
  c=transition_prob.shape[1]
  for i in range(r):
    for j in range(r):
      tp[labels[i],labels[j]]=transition_prob[i][j]
  ep=cls.defaultdict(float)
  for i in range(r):
    for j in range(len(alphabet)):
      ep[labels[i],alphabet[j]]=emission_prob[i][j]

# Graph creation
  graph=cls.defaultdict(list)
  initial_states=["I0","M1","D1"]
# Initial state transitions
  for next_state in initial_states:
    graph["S"].append({"node": next_state,"weight":tp["S",next_state]})
# Internal State transitions
  for i in range(n):
    current_state=f"I{i}"
    states=[f"I{i}",f"M{i+1}",f"D{i+1}"]
    for next_state in states:
      graph[current_state].append({"node": next_state,"weight":tp[current_state,next_state]})
# Transition between matched and deleted state
  for i in range(1,n):
    current_states=[f"M{i}",f"D{i}"]
    next_states=[f"I{i}",f"M{i+1}",f"D{i+1}"]
    for cs in current_states:
      for ns in next_states:
        graph[cs].append({"node": ns,"weight":tp[cs,ns]})
# Last state
  final_states=[f"I{n}",f"M{n}",f"D{n}"]
  for state in final_states:
    graph[state].append({"node": f"I{n}","weight":tp[state,f"I{n}"]})
    graph[state].append({"node": "E","weight":tp[state,"E"]})
  m=len(text)
  # storing topological ordering in result
  result=[("S",0)]
  # transitions from deletions
  for i in range(n):
    result.append((f"D{i+1}",0))
  for i in range(m):
    result.append(("I0",i+1))
    for j in range(n):
      for cat in ["M","D","I"]:
        result.append((f"{cat}{j+1}",i+1))
  # end state
  result.append(("E",i+2))
  output={result[0]:0} #probability of hidden states
  pointer={result[0]:(None,None)} # stroing path of states observed
  result=result[1:]

  # Changing graph into node weight type of dictionary for ease of computation
  g={}
  for a,b in graph.items():
    nodes={}
    for t in b:
      nodes[t["node"]]=t["weight"]
    g[a]=nodes
  graph=g

  # Iterate in topological order
  for no,c in result:
    pointer[(no,c)]=0
    output[(no,c)]=-inf
    previous_nodes=backtracking(n,m,no,c) #finding previous states using backtracking one step

    for pn,pc in previous_nodes:

      if pc < c and no!="E":
        emission=ep[no,text[c-1]]
      else:
        emission=1
      # try:
      if graph[pn][no]==0:
        graph[pn][no]=0.00000001 #avoiding zero errors in log
      if emission==0:
        emission=0.00000001 #avoid zero error in log
      prob=log(graph[pn][no])+log(emission)+output[(pn,pc)]  #update the path probability incase if having higher probability
      if prob>output[(no,c)]: # maximizing the probability of hidden states to find the optimal path
        output[(no,c)]=prob #update the hidden states prob
        pointer[(no,c)]=(pn,pc) # update the hidden states path

  path=[]
  end=("E",len(text)+1)

  while end[0]!=None:
    path.append(pointer[end][0]) #finding and storing the path by backtracing
    end=pointer[end]

  path.reverse()
  path=path[2:] #neglecting none and start state
  print("Path:",end=" ")
  for i in path:
    print(i,end=" ")


alphabet="ABCDEF"
alphabet=alphabet.replace(" ","")
alphabet=[char for char in alphabet]

alignment = [
    "ACDEFACADF",
    "AFDA---CCF",
    "A--EFD-FDC",
    "ACAEF--A-C",
    "ADDEFAAADF"
]

text="AEFDFDC"

theta=0.4
sigma=0.01


alignment=[list([s for s in seq]) for seq in alignment]

q3(text,theta,sigma,alignment,alphabet)

Path: M1 D2 D3 M4 M5 I5 M6 M7 M8 